In [1]:
import sys, os
sys.path.append('./glm_10b_chinese')

import torch
import transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import deepspeed

from modeling_glm_cache_pruning_prompt import GLMForConditionalGeneration
# from modeling_glm_cache import GLMBlock

# from modeling_glm import GLMForConditionalGeneration
from tokenization_glm import GLMChineseTokenizer

In [3]:
tokenizer = GLMChineseTokenizer.from_pretrained('./glm_10b_chinese')

In [24]:
prompt = '请介绍一下GLM:\n'
input_txt = '十五个字以内[gMASK]'

prompt_tokens = tokenizer(prompt, return_tensors='pt')
input_tokens = tokenizer(input_txt, return_tensors='pt')
prompt_tokens,input_tokens

({'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383, 50000]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[50002, 35197, 19305,  6490, 50009, 50000]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])})

In [33]:
all_tokens = tokenizer(prompt+input_txt, return_tensors='pt')
all_tokens

{'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383, 35197, 19305,  6490, 50009,
         50000]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [26]:
def post_process(inputs, ignore_first=True):
    keys = ['input_ids', 'token_type_ids', 'attention_mask']
    data = {}
    for k in keys:
        if ignore_first:
            data[k] = inputs[k][:,1:]
        else:
            data[k] = inputs[k][:,:-1]
    return BatchEncoding(data=data)

In [27]:
prompt_tokens = post_process(prompt_tokens, ignore_first=False)
input_tokens = post_process(input_tokens, ignore_first=True)

prompt_tokens, input_tokens

({'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[35197, 19305,  6490, 50009, 50000]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])})

In [15]:
type(prompt_tokens)

transformers.tokenization_utils_base.BatchEncoding

In [21]:
from tokenization_glm import BatchEncoding

BatchEncoding(data={'a':'a'})['a']

'a'

In [34]:
all_tokens

{'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383, 35197, 19305,  6490, 50009,
         50000]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [36]:
g_tokens = tokenizer.build_inputs_for_generation(all_tokens)
g_tokens

{'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383, 35197, 19305,  6490, 50009,
         50000, 50006]]), 'position_ids': tensor([[[  0,   1,   2,  ...,   9,   9,   9],
         [  0,   0,   0,  ..., 510, 511, 512]]]), 'generation_attention_mask': tensor([[[[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 1, 0, 0],
          [1, 1, 1,  ..., 1, 1, 0],
          [1, 1, 1,  ..., 1, 1, 1]]]])}

In [40]:
g_tokens.position_ids[0][0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9, 

In [41]:
g_tokens.position_ids[0][1]

tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   2,   3,
          4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
         18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
         32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
         46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
         60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
         74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
        102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
        158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 1

In [38]:
g_tokens.generation_attention_mask.size()

torch.Size([1, 1, 523, 523])

In [50]:
prompt = '请介绍一下GLM:\n'
input_txt = '[gMASK]十五个字以内[gMASK]'
all_tokens = tokenizer(prompt+input_txt, return_tensors='pt')
all_tokens

{'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383, 50009, 35197, 19305,  6490,
         50009, 50000]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [58]:
prompt = '请介绍一下GLM:\n'
input_txt = '十五个字以内[gMASK]'
all_tokens = tokenizer(prompt+input_txt, return_tensors='pt')
all_tokens

{'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383, 35197, 19305,  6490, 50009,
         50000]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [59]:
tokenizer.batch_decode(all_tokens.input_ids)

['[CLS] 请介绍一下GLM: 十五个字以内 [gMASK] <|endoftext|>']

In [60]:
g_tokens = tokenizer.build_inputs_for_generation(all_tokens)
g_tokens

{'input_ids': tensor([[50002,  2118, 17178, 20640, 43638, 43383, 35197, 19305,  6490, 50009,
         50000, 50006]]), 'position_ids': tensor([[[  0,   1,   2,  ...,   9,   9,   9],
         [  0,   0,   0,  ..., 510, 511, 512]]]), 'generation_attention_mask': tensor([[[[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 1, 0, 0],
          [1, 1, 1,  ..., 1, 1, 0],
          [1, 1, 1,  ..., 1, 1, 1]]]])}

In [61]:
g_tokens.input_ids.size()


torch.Size([1, 12])

In [62]:
g_tokens.position_ids.size()

torch.Size([1, 2, 523])

In [63]:
g_tokens.generation_attention_mask.size()

torch.Size([1, 1, 523, 523])